This is coded on an ESP8266 on the coffee level galvonometer
now hanging on the pipework at the front.

Very trivial code, no checking or async work

In [9]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [10]:
%sendtofile main.py

import machine, time, network

wifiname = "mqtt-wifi"
wifipassword = "decafbad00"
mqttbroker = "10.3.141.1"

# nodemcu8266 D1 = pin5, D2 = pin4; use pin13 on an esp32
p = machine.PWM(machine.Pin(5))  
p.duty(1023)
time.sleep(0.5)

sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)

p.duty(0)
sta_if.connect(wifiname, wifipassword)
while not sta_if.isconnected():
    time.sleep(0.1)
p.duty(1023)

print("connected to wifi")
from umqtt.robust import MQTTClient
client = MQTTClient("coffeedial", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
    p.duty(1023*(i%2))

print("connected to mqtt")

def sub_cb(topic, msg):
    print((topic, msg))
    try:
        v = float(msg)
    except ValueError:
        print("ignore")
        return
    v = min(100, max(0, v))
    p.duty(int(v/100*1023))

client.set_callback(sub_cb)
client.subscribe(b"coffeedial/level")
p.duty(500)

while True:
    client.wait_msg()


Sent 49 lines (995 bytes) to main.py.


In [8]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-8-ga9a3caad0 on 2018-05-11; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f5388052358, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [6]:
print(p)

PWM(5, freq=500, duty=1023)


In [158]:

import machine
p = machine.PWM(machine.Pin(5))
# nodemcu D1 = pin5, D2 = pin4


In [184]:
# this makes a very rapid move to the spot
# without the wobbling
# (could be implemented, and then use schedule to do the callback)
import time

p.duty(0)
time.sleep(1)

p.duty(1023)
time.sleep(0.09)
p.duty(0)
time.sleep(0.1)
p.duty(500)




.

In [235]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [236]:
import machine
a = machine.ADC(0)
p = machine.Pin(5, machine.Pin.OUT)


In [247]:
import time
p.value(1)
time.sleep_ms(50)
print(a.read())
p.value(0)
time.sleep_ms(50)
print(a.read())


212
9


In [234]:
p.value(0)
time.sleep_ms(50)
print(a.read())




***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fa00504b588, open=True>(port='/dev/ttyUSB3', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands